In [383]:
import logging
import pandas as pd
import numpy as np
import os
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy import get_logger
from yfpy.utils import (
    unpack_data,
    dict_to_list,
    complex_json_handler,
    flatten_json_dict_list,
    flatten_to_list,
    reformat_json_list,
)
from dotenv import load_dotenv
from gspread_pandas import Spread

from utils import (
    get_season,
    league_season_info,
    google_sheet_upload,
    sql_upload_table,
    sql_grab_table,
)


load_dotenv()
pd.set_option("display.max_colwidth", 999)
pd.set_option("display.max_columns", 999)

LOGGER = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

PATH = Path.cwd().parents[0]
DATA_DIR = PATH / "data"

SEASON = get_season()
NFL_DATES_DF, LEAGUE_ID_DF = league_season_info()

# TODAY = np.datetime64("today", "D")
TODAY = np.datetime64("2021-09-28")
NFL_WEEK = NFL_DATES_DF["Week"][
    (NFL_DATES_DF["End_Date"] >= TODAY) & (NFL_DATES_DF["Start_Date"] <= TODAY)
].values[0]
LEAGUE_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["league_ID"].values[0]
GAME_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["game_ID"].values[0]
WEEKS = list(range(NFL_WEEK, 0, -1))

CONSUMER_KEY = os.getenv("yahoo_client_id")
CONSUMER_SECRET = os.getenv("yahoo_client_secret")

try:
    yahoo_query = YahooFantasySportsQuery(
        auth_dir=PATH,
        league_id=LEAGUE_ID,
        game_id=GAME_ID,
        game_code="nfl",
        offline=False,
        all_output_as_json=False,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        browser_callback=True,
    )


except Exception as e:
    print(e)

In [331]:
response = yahoo_query.get_league_teams()
teams = pd.DataFrame()
for r in response:
    team = pd.json_normalize(complex_json_handler(r['team']))
    manager = pd.json_normalize(complex_json_handler(team['managers.manager'][0]))
    team = pd.concat([team, manager], axis=1)
    if 'clinched_playoffs' in team.columns:
        teams = pd.concat([teams, team], ignore_index=True).reset_index(drop=True)
        teams.drop(['url', 'managers.manager', 'team_logos.team_logo', 'image_url'], axis=1, inplace=True)
    else:
        team['clinched_playoffs'] = 0
        teams = pd.concat([teams, team], ignore_index=True).reset_index(drop=True)
        teams.drop(['url', 'managers.manager', 'team_logos.team_logo', 'image_url'], axis=1, inplace=True)
teams['name'] = teams['name'].str.decode("utf-8")

,clinched_playoffs,draft_grade,draft_recap_url,faab_balance,has_draft_grade,league_scoring_type,name,number_of_moves,number_of_trades,team_id,team_key,waiver_priority,roster_adds.coverage_type,roster_adds.coverage_value,roster_adds.value,guid,manager_id,nickname
0,1,B,https://football.fantasysports.yahoo.com/2021/f1/103661/1/draftrecap,0,1,head,Fantasy Sidelines,73,2,1,406.l.103661.t.1,8,week,18,0,O7GVEWK7WENDLVWC5A5HD2IJ5Y,1,Tim
1,1,B-,https://football.fantasysports.yahoo.com/2021/f1/103661/2/draftrecap,0,1,head,Dak to the Future,30,2,2,406.l.103661.t.2,1,week,18,0,55AOAC6JFAY2M2MFV7RCYOSMYI,2,Wesley
2,0,B,https://football.fantasysports.yahoo.com/2021/f1/103661/3/draftrecap,0,1,head,Fightin’ Blue Cock/Chubb/Wood,35,0,3,406.l.103661.t.3,10,week,18,0,NURTYROM34YVSDV7DFZNDBPMAY,3,Kevin
3,1,A-,https://football.fantasysports.yahoo.com/2021/f1/103661/4/draftrecap,0,1,head,King of the Hill,54,0,4,406.l.103661.t.4,2,week,18,0,WIFGX5C4WY7VCFB5ZYEG65WFRE,4,ryan
4,0,A-,https://football.fantasysports.yahoo.com/2021/f1/103661/5/draftrecap,60,1,head,The Titanic,12,0,5,406.l.103661.t.5,4,week,18,0,K5XZK5B6DWAD5X5EJKI2OCNN4A,5,Jeremy
5,0,C-,https://football.fantasysports.yahoo.com/2021/f1/103661/6/draftrecap,0,1,head,Wide Out University,37,0,6,406.l.103661.t.6,6,week,18,0,QCTL3IHSUQZ634KEJTBJW5AZJM,6,Chris
6,1,C,https://football.fantasysports.yahoo.com/2021/f1/103661/7/draftrecap,34,1,head,Harris & Harris RBs at Law 👨🏾‍⚖️,53,0,7,406.l.103661.t.7,7,week,18,0,XPS6HEP4KQOOGAG2HIGIJ6ZWEM,7,Greg
7,0,C+,https://football.fantasysports.yahoo.com/2021/f1/103661/8/draftrecap,56,1,head,Poor Decisions 😢,32,1,8,406.l.103661.t.8,9,week,18,0,L3DNOFNKTGQ646BS6QOM7GTJNY,8,Peter Billups
8,0,C,https://football.fantasysports.yahoo.com/2021/f1/103661/9/draftrecap,28,1,head,Push Ups in Prison,25,0,9,406.l.103661.t.9,3,week,18,0,4W75HRWQNEMHR4WCIUUDRJZV6A,9,Pat
9,0,A,https://football.fantasysports.yahoo.com/2021/f1/103661/10/draftrecap,44,1,head,Team 10,43,1,10,406.l.103661.t.10,5,week,18,0,SUIVCBYJ5MSGEFDRCTCP7CK45M,10,James


In [42]:
class league_season_data(object):
    def __init__(
        self,
        auth_dir: Path,
        league_id: str,
        game_id: int = None,
        game_code: str = "nfl",
        offline: bool = False,
        all_output_as_json: bool = False,
        consumer_key: str = None,
        consumer_secret: str = None,
        browser_callback: bool = True,
    ):

        self._auth_dir = auth_dir
        self._consumer_key = consumer_key
        self._consumer_secret = consumer_secret
        self._browser_callback = browser_callback

        self.league_id = league_id
        self.game_id = game_id
        self.game_code = game_code

        self.offline = offline
        self.all_output_as_json = all_output_as_json

    def _yahoo_query(self):

        try:
            yahoo_query = YahooFantasySportsQuery(
                auth_dir=self._auth_dir,
                league_id=self.league_id,
                game_id=self.game_id,
                game_code=self.game_code,
                offline=self.offline,
                all_output_as_json=self.all_output_as_json,
                consumer_key=self._consumer_key,
                consumer_secret=self._consumer_secret,
                browser_callback=self._browser_callback,
            )

            return yahoo_query

        except Exception as e:
            print(e)

    def get_league_metadata(self):
        response = complex_json_handler(yahoo_query.get_league_metadata())
        league_metadata =  pd.json_normalize(response)
        league_metadata['game_id'] = self.game_id

        return league_metadata

    def get_league_set_roster_pos_stat_cat(self):
        response = complex_json_handler(yahoo_query.get_league_settings())

        league_settings = pd.json_normalize(response)
        league_settings.drop(['roster_positions', 'stat_categories.stats', 'stat_modifiers.stats'], axis=1, inplace=True)
        league_settings['game_id'] = self.game_id
        league_settings['league_id'] = self.league_id

        roster_positions = pd.DataFrame()
        for r in response['roster_positions']:
            row = pd.json_normalize(complex_json_handler(r['roster_position']))
            roster_positions = pd.concat([roster_positions, row])
        roster_positions['game_id'] = self.game_id
        roster_positions['league_id'] = self.league_id

        stat_categories = pd.DataFrame()
        for r in response['stat_categories']['stats']:
            row = pd.json_normalize(complex_json_handler(r['stat']))
            row['position_type'] = complex_json_handler(complex_json_handler(r['stat'])['stat_position_types']['stat_position_type'])['position_type']
            try:
                row['is_only_display_stat'] = complex_json_handler(complex_json_handler(r['stat'])['stat_position_types']['stat_position_type'])['is_only_display_stat']
            except:
                row['is_only_display_stat'] = 0
            row.drop('stat_position_types.stat_position_type', axis=1, inplace=True)
            stat_categories = pd.concat([stat_categories, row])
        stat_categories['game_id'] = self.game_id
        stat_categories['league_id'] = self.league_id

        stat_modifiers = pd.DataFrame()
        for r in response['stat_modifiers']['stats']:
            row = pd.json_normalize(complex_json_handler(r['stat']))
            stat_modifiers = pd.concat([stat_modifiers, row])
        stat_modifiers.rename(columns={'value': 'stat_modifier'}, inplace=True)

        stat_categories = stat_categories.merge(stat_modifiers, how='outer', on='stat_id')

        return league_settings, roster_positions, stat_categories

    def get_league_teams_list(self):
        response = yahoo_query.get_league_teams()
        teams = pd.DataFrame()
        for r in response:
            team = pd.json_normalize(complex_json_handler(r['team']))
            manager = pd.json_normalize(complex_json_handler(team['managers.manager'][0]))
            team = pd.concat([team, manager], axis=1)
            if 'clinched_playoffs' in team.columns:
                teams = pd.concat([teams, team], ignore_index=True).reset_index(drop=True)
                teams.drop(['url', 'managers.manager', 'team_logos.team_logo', 'image_url'], axis=1, inplace=True)
            else:
                team['clinched_playoffs'] = 0
                teams = pd.concat([teams, team], ignore_index=True).reset_index(drop=True)
                teams.drop(['url', 'managers.manager', 'team_logos.team_logo', 'image_url'], axis=1, inplace=True)
        teams['name'] = teams['name'].str.decode("utf-8")
        teams['game_id'] = self.game_id
        teams['league_id'] = self.league_id

        return teams

    def get_league_players_list(self):
        response = yahoo_query.get_league_players()
        players = pd.DataFrame()
        for r in response:
            player = pd.json_normalize(complex_json_handler(r['player']))
            players = pd.concat([players, player])
        players['game_id'] = self.game_id
        players['league_id'] = self.league_id

        return players

    def get_league_draft_results(self):
        response = yahoo_query.get_league_draft_results()
        draft_results = pd.DataFrame()
        for r in response:
            row = pd.json_normalize(complex_json_handler(r['draft_result']))
            draft_results = pd.concat([draft_results, row])
        draft_results['game_id'] = self.game_id
        draft_results['league_id'] = self.league_id

        return draft_results

    def get_league_matchups_by_week(self):
        m = []
        i = 0
        while True:
            try:
                response = yahoo_query.get_league_matchups_by_week(i)
                if bool(response) == True:
                    for data in response:
                        m.append(complex_json_handler(data['matchup']))
                    i += 1
                else:
                    break
            except:
                break
        matchups = pd.DataFrame()
        for r in m:
            matchup = pd.json_normalize(r)
            team_a = pd.json_normalize(complex_json_handler(r['matchup_grades'][0]['matchup_grade']))
            team_a['points'] = complex_json_handler(r['teams'][0]['team'])['team_points']['total']
            team_a['projected_points'] = complex_json_handler(r['teams'][0]['team'])['team_projected_points']['total']
            team_a = team_a.add_prefix('team_a_')
            team_b = pd.json_normalize(complex_json_handler(r['matchup_grades'][1]['matchup_grade']))
            team_b['points'] = complex_json_handler(r['teams'][1]['team'])['team_points']['total']
            team_b['projected_points'] = complex_json_handler(r['teams'][1]['team'])['team_projected_points']['total']
            team_b = team_b.add_prefix('team_b_')
            matchup = pd.concat([matchup, team_a, team_b], axis=1)

            matchups = pd.concat([matchups, matchup])

        matchups.drop(['teams', 'matchup_grades'], axis=1, inplace=True)
        matchups['game_id'] = self.game_id
        matchups['league_id']  = self.league_id

        return matchups

    def get_team_standings(self):
        pass

    def get_team_roster_by_week(self):
        pass

    def get_player_stats_by_week(self):
        pass

    def get_player_percent_owned_by_week(self):
        pass

    def get_player_draft_analysis(self):
        pass


    def all_game_keys(self):
        response = unpack_data(yahoo_query.get_all_yahoo_fantasy_game_keys())
        game_keys = pd.DataFrame()
        for r in response:
            row = pd.DataFrame(complex_json_handler(r["game"]), index=[0])
            game_keys = pd.concat([game_keys, row])
        game_keys.reset_index(drop=True, inplace=True)
        game_keys = game_keys[game_keys["season"] >= 2012]
        sql_upload_table(
            dataframe=game_keys,
            table_name="GameKeys",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )


    def all_nfl_weeks(self):
        game_keys = sql_grab_table("GameKeys")
        game_id = list(game_keys["game_id"])
        weeks = pd.DataFrame()
        for g in game_id:
            response = yahoo_query.get_game_weeks_by_game_id(g)
            for r in response:
                row = pd.json_normalize(complex_json_handler(r["game_week"]))
                row["game_id"] = g
                weeks = pd.concat([weeks, row])
        weeks.rename(columns={"display_name": "week"}, inplace=True)
        sql_upload_table(
            dataframe=weeks,
            table_name="NFLWeeks",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )